### LLM

In [1]:
import os
from zhipu import ChatGLM
chatglm = ChatGLM(api_key=os.environ["ZHIPU_API_KEY"])

In [2]:
resp = chatglm("你好")
print(resp)

你好👋！我是人工智能助手智谱清言，可以叫我小智🤖，很高兴见到你，欢迎问我任何问题。


### chat model

In [3]:
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage

chat_model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

text = "你好"
messages = [HumanMessage(content=text)]
resp = chat_model.invoke(messages)
resp

2023-12-21 10:53:20 [INFO][_client.py:1027]:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='你好！有什么我可以帮助你的吗？')

In [7]:
print(resp.content)

你好！有什么我可以帮助你的吗？


### prompt template

In [4]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")


[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

### Output parser

In [5]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")


['hi', 'bye']

### Composing with LCEL


In [6]:
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage

chat_model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0)

In [8]:
from langchain.prompts.chat import ChatPromptTemplate

template = """你是一个知识渊博的助手，请将我的问题按照list格式输出，list中的每个元素都是一个json，需要包含如下字段:
{fields}
"""
human_template = "请列出三个最出名的{country}人"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])




In [9]:
from typing import List
class JsonParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        print(text)

        info = eval(text)
        for item in info:
            for k,v in item.items():
                print(f"{k}:{v}")
            print("\n\n")
        
json_parser = JsonParser()

In [10]:
chain = chat_prompt | chat_model | json_parser

In [12]:
data = dict(country="德国", fields="\n".join(["姓名", "出生年月", "职业"]))

resp=chain.invoke(data)




2023-12-21 10:57:04 [INFO][_client.py:1027]:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[
  {
    "姓名": "阿尔伯特·爱因斯坦",
    "出生年月": "1879年3月14日",
    "职业": "物理学家"
  },
  {
    "姓名": "约翰·巴赫",
    "出生年月": "1685年3月31日",
    "职业": "作曲家"
  },
  {
    "姓名": "安格拉·默克尔",
    "出生年月": "1954年7月17日",
    "职业": "政治家"
  }
]
姓名:阿尔伯特·爱因斯坦
出生年月:1879年3月14日
职业:物理学家



姓名:约翰·巴赫
出生年月:1685年3月31日
职业:作曲家



姓名:安格拉·默克尔
出生年月:1954年7月17日
职业:政治家



